In [1]:
import pandas as pd
import time

In [2]:
# Decision Tree(C&RT) with Gini Impurity

class DTree(object):
    def __init__(self, d=2):
        # feature dimension of the input
        self.dim = d
        # Record where and how to branch
        self.Branch = {}
        # Record the predict values at leaves
        self.Value = {}

    def build_tree(self, df, layer=0, side=0, max_depth=100):
        """Build the decision tree by recursively branching by Gini impurity"""
        # Cannot be branched anymore
        if len(set(df.y)) == 1:
            self.Value[(layer, side)] = df.y.values[0]
        # Depth reaches the limit
        elif layer >= max_depth:
            self.Value[(layer, side)] = 2 * (sum(df.y.values) >= 0) - 1
        else:
            best_d, best_val = self.branching(df, layer, side)
            # Left hand side
            p = (df[df[best_d] >= best_val], layer + 1, 2 * side, max_depth)
            self.build_tree(*p)
            # Right hand side
            p = (df[df[best_d] < best_val], layer + 1, 2 * side + 1, max_depth)
            self.build_tree(*p)

    def branching(self, df, layer, side):
        """find the value of i-th feature for the best branching"""
        min_err = 1
        # Search for the best cut
        for i in range(self.dim):
            ddf = df.sort_values(i)
            Y = ddf.y.values

            for j in range(1, len(ddf)):
                err = self.impurity(Y, j)
                if err < min_err:
                    best_d, best_val, min_err = i, ddf.iloc[j][i], err

        # Record the best branching parameters at this node
        self.Branch[(layer, side)] = best_d, best_val
        return best_d, best_val

    def impurity(self, Y, j):
        """Gini impurity for binary classification"""
        # Neglect repeated entries
        if Y[j] == Y[j-1]:
            return 1

        Y1 = sum(Y[:j])
        Y2 = sum(Y[j:])
        N = len(Y)
        T1 = j**2 - (Y1)**2
        T2 = (N - j)**2 - (Y2)**2

        return (T1 / j + T2 / (N - j)) / N

    def predict(self, X, layer=0, side=0):
        """recursively traversing: Predict which class the input belongs to"""
        if (layer, side) in self.Value:
            return self.Value[(layer, side)]
        else:
            branch_d, branch_val = self.Branch[(layer, side)]
            C = 0 if X[branch_d] >= branch_val else 1
            return self.predict(X, layer + 1, 2 * side + C)

In [3]:
# Loading Data
train_data = pd.read_csv('Data/hw3_train.dat', sep=' ', header=None, names=[0, 1, 'y'])
train_data.head()

,0,1,y
0,0.757222,0.633831,-1
1,0.847382,0.281581,-1
2,0.249310,0.618635,1
3,0.538526,0.144259,-1
4,0.474435,0.414558,-1


In [4]:
test_data = pd.read_csv('Data/hw3_test.dat', sep=' ', header=None, names=[0, 1, 'y'])
test_data.head()

,0,1,y
0,0.984250,0.712610,-1
1,0.901491,0.462824,-1
2,0.872418,0.365547,-1
3,0.810913,0.058338,-1
4,0.577230,0.203007,-1


In [5]:
# Train on 1 decision tree

Tree = DTree()
Tree.build_tree(train_data)
Tree_Start = time.clock()

print("Train on 1 fully-grown Decision Tree.\nBranches:")
for k in sorted(Tree.Branch.items()):
    print('\t', k)

train_pred = [Tree.predict(X) for X in train_data[[0, 1]].values]
train_accu = sum(train_pred == train_data.y) * 100 / len(train_data)
print("\tAccuracy on Train set: %.3f %%" % train_accu)

test_pred = [Tree.predict(X) for X in test_data[[0, 1]].values]
test_accu = sum(test_pred == test_data.y) * 100 / len(test_data)
print("\tAccuracy on Test set: %.3f %%" % test_accu)
print("Using %.3f seconds" % (time.clock() - Tree_Start))

Train on 1 fully-grown Decision Tree.
Branches:
	 ((0, 0), (1, 0.63383100000000003))
	 ((1, 0), (0, 0.89615300000000009))
	 ((1, 1), (0, 0.24362500000000001))
	 ((2, 2), (0, 0.54448999999999992))
	 ((2, 3), (1, 0.178643))
	 ((3, 4), (1, 0.290269))
	 ((3, 5), (1, 0.41455799999999998))
	 ((4, 9), (1, 0.28158099999999997))
	 ((4, 10), (0, 0.27219299999999996))
	 ((4, 11), (0, 0.53852600000000006))
	Accuracy on Train set: 100.000 %
	Accuracy on Test set: 90.600 %
Using 0.015 seconds
